In [ ]:
#using a public API, Coin Market Cap. 

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '32d058ab-7933-46fe-8c0f-4119636488fd',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
import pandas as pd

#This allows you to see all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.json_normalize(data['data'])

In [ ]:
#This normalizes the data.

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

In [ ]:
#Create a function using the original script, appending data to the dataframe.
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    # Just keep it in a dataframe
    
    df2 = pd.json_normalize(data['data'])
    df2['Timestamp'] = pd.to_datetime('now')
    df = df.append(df2)
    
    # Create a csv and append data to it
    
    #df = pd.json_normalize(data['data'])
    #df['Timestamp'] = pd.to_datetime('now')
    #df
    
    #if not os.path.isfile(r'C:\Users\To\OneDrive\Documentos\API.csv'):
        #df.to_csv(r'C:\Users\To\OneDrive\Documentos\API.csv', header = 'column_names')
    #else:
        #df.to_csv(r'C:\Users\To\OneDrive\Documentos\API.csv', mode = 'a', header = False)
        

In [ ]:
#call the function in intervals of time.
import os
from time import time
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner completed')
    sleep(60) #sleep for one minute.
exit()

In [ ]:
df84 = pd.read_csv(r'C:\Users\To\OneDrive\Documentos\API.csv')
df84

In [ ]:
df


In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df

In [ ]:
# Coin trends over time

df3 = df.groupby('name', sort = False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df3

In [ ]:
df4 = df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
df5 = df4.to_frame(name='values')
df5

In [ ]:
df5.count()


In [ ]:
#Create a range and pass that as the dataframe
index = pd.Index(range(90))

# Set the above DataFrame index object as the index
# using set_index() function
df6 = df5.reset_index()
df6

In [ ]:
# Change the column name
df7 = df6.rename(columns={'level_1': 'percentage_change'})  
df7

In [ ]:
df7['percentage_change'] = df7['percentage_change'].replace(['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])
df7

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.catplot(x ='percentage_change', y = 'values', hue = 'name', data = df7, kind = 'point')

In [ ]:
#create a dataframe with the some columns
df9 = df[['name','quote.USD.price','Timestamp']]
df9 = df9.query("name == 'Bitcoin'")
df9

In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x='Timestamp', y='quote.USD.price', data = df9)